# Intuitional Description

Genotype coding converts DNA nucleotide pairs into numerical values for statistical analysis, with the additive model being particularly valuable because genetic effects often accumulate proportionally with each additional copy of a variant.


# Graphical Summary

# Key Formula

We use $\mathbf{X}_\text{raw}$ to denote the **Raw Genotype Matrix** (an $N \times M$ matrix representing genotypes for $N$ individuals at $M$ variants)
   
For diploid organisms, using different coding models:
   - Additive model: $x_{ij} \in \{0, 1, 2\}$ (count of alternative alleles)
   - Dominant model: $x_{ij} \in \{0, 1\}$ (1: presence of alternative allele)
   - Recessive model: $x_{ij} \in \{0, 1\}$ (two copies of alternative allele required for 1)

**Standardized Genotype Matrix** $\mathbf{X}$: Normalized version of $\mathbf{X}_\text{raw}$

For each column $j$ (variant):
   $$X_{ij} = \frac{X_{\text{raw},ij} - \mu_j}{\sigma_j}$$
   
   Where:
   - $\mu_j = \frac{1}{N}\sum_{i=1}^{N} X_{\text{raw},ij}$ (mean of variant $j$)
   - $\sigma_j = \sqrt{\frac{1}{N}\sum_{i=1}^{N} (X_{\text{raw},ij} - \mu_j)^2}$ (standard deviation of variant $j$)

# Technical Details

## Beyond SNPs: Encoding Different Variant Types

While we commonly encode SNPs, the same principles apply to other genetic variants:

1. **Insertions and Deletions (Indels)**
  - Small insertions or deletions in DNA sequence
  - Typically encoded using the same 0/1/2 scheme as SNPs

2. **Copy Number Variations (CNVs)**
  - Deletions or duplications of larger DNA segments
  - Can be encoded as actual copy number (0, 1, 2, 3, etc.)

3. **Structural Variants**
  - Inversions, translocations, and complex rearrangements
  - Often encoded as binary (presence/absence)

The choice of encoding scheme should reflect the biological hypothesis about how the variant affects the phenotype.

# Example

Let's illustrate how we encode genotypes using different models and standardize them. We'll start with the raw allele counts and demonstrate the process step by step.

In [9]:
# Clear the environment
rm(list = ls())

# Define genotypes for 5 individuals at 3 variants
# These represent actual alleles at each position
# For example, Individual 1 has genotypes: CC, CT, AT
genotypes <- c(
 "CC", "CT", "AT",  # Individual 1
 "TT", "TT", "AA",  # Individual 2
 "CT", "CT", "AT",  # Individual 3
 "CC", "TT", "AT",  # Individual 4
 "CC", "CC", "TT"   # Individual 5
)
# Reshape into a matrix
geno_matrix <- matrix(genotypes, nrow=5, ncol=3, byrow=TRUE)
rownames(geno_matrix) <- paste("Individual", 1:5)
colnames(geno_matrix) <- paste("Variant", 1:3)
print("Raw genotypes:")
geno_matrix


[1] "Raw genotypes:"


,Variant 1,Variant 2,Variant 3
Individual 1,CC,CT,AT
Individual 2,TT,TT,AA
Individual 3,CT,CT,AT
Individual 4,CC,TT,AT
Individual 5,CC,CC,TT


In [10]:
# Define reference alleles for each variant
# Let's assume the reference alleles are:
ref_alleles <- c("C", "C", "A")
names(ref_alleles) <- colnames(geno_matrix)
print("Reference alleles:")
print(ref_alleles)

[1] "Reference alleles:"
Variant 1 Variant 2 Variant 3 
      "C"       "C"       "A" 


In [11]:
# Convert to raw genotype matrix using the additive / dominant / recessive model
Xraw_additive <- matrix(0, nrow=nrow(geno_matrix), ncol=ncol(geno_matrix)) # dount number of non-reference alleles
Xraw_dominant <- matrix(0, nrow=nrow(geno_matrix), ncol=ncol(geno_matrix)) # presence of alternative allele
Xraw_recessive <- matrix(0, nrow=nrow(geno_matrix), ncol=ncol(geno_matrix)) # two copies of alternative allele required

rownames(Xraw_additive) <- rownames(Xraw_dominant) <- rownames(Xraw_recessive) <- rownames(geno_matrix)
colnames(Xraw_additive) <- colnames(Xraw_dominant) <- colnames(Xraw_recessive) <- colnames(geno_matrix)

for (i in 1:nrow(geno_matrix)) {
  for (j in 1:ncol(geno_matrix)) {
    alleles <- strsplit(geno_matrix[i,j], "")[[1]]
    Xraw_additive[i,j] <- sum(alleles != ref_alleles[j])
    Xraw_dominant[i,j] <- as.integer(any(alleles != ref_alleles[j]))
    Xraw_recessive[i,j] <- as.integer(all(alleles != ref_alleles[j]))
  }
}


In [12]:
print("Raw genotype matrix (Additive model: count of alternative alleles):")
print(Xraw_additive)

[1] "Raw genotype matrix (Additive model: count of alternative alleles):"
             Variant 1 Variant 2 Variant 3
Individual 1         0         1         1
Individual 2         2         2         0
Individual 3         1         1         1
Individual 4         0         2         1
Individual 5         0         0         2


In [13]:
print("Dominant model matrix (1 if any alternative allele is present):")
print(Xraw_dominant)

[1] "Dominant model matrix (1 if any alternative allele is present):"
             Variant 1 Variant 2 Variant 3
Individual 1         0         1         1
Individual 2         1         1         0
Individual 3         1         1         1
Individual 4         0         1         1
Individual 5         0         0         1


In [14]:
print("Recessive model matrix (1 if both alleles are alternative):")
print(Xraw_recessive)

[1] "Recessive model matrix (1 if both alleles are alternative):"
             Variant 1 Variant 2 Variant 3
Individual 1         0         0         0
Individual 2         1         1         0
Individual 3         0         0         0
Individual 4         0         1         0
Individual 5         0         0         1


In [15]:
# Standardize each raw genotype matrix
X_additive <- scale(Xraw_additive, center=TRUE, scale=TRUE)
X_dominant <- scale(Xraw_dominant, center=TRUE, scale=TRUE)
X_recessive <- scale(Xraw_recessive, center=TRUE, scale=TRUE)

In [16]:
print("Standardized genotype matrix (Additive model):")
X_additive

[1] "Standardized genotype matrix (Additive model):"


,Variant 1,Variant 2,Variant 3
Individual 1,-0.6708204,-0.2390457,0.000000
Individual 2,1.5652476,0.9561829,-1.414214
Individual 3,0.4472136,-0.2390457,0.000000
Individual 4,-0.6708204,0.9561829,0.000000
Individual 5,-0.6708204,-1.4342743,1.414214


In [17]:
print("Standardized genotype matrix (Dominant model):")
X_dominant

[1] "Standardized genotype matrix (Dominant model):"


,Variant 1,Variant 2,Variant 3
Individual 1,-0.7302967,0.4472136,0.4472136
Individual 2,1.0954451,0.4472136,-1.7888544
Individual 3,1.0954451,0.4472136,0.4472136
Individual 4,-0.7302967,0.4472136,0.4472136
Individual 5,-0.7302967,-1.7888544,0.4472136


In [19]:
print("Standardized genotype matrix (Recessive model):")
X_recessive

[1] "Standardized genotype matrix (Recessive model):"


,Variant 1,Variant 2,Variant 3
Individual 1,-0.4472136,-0.7302967,-0.4472136
Individual 2,1.7888544,1.0954451,-0.4472136
Individual 3,-0.4472136,-0.7302967,-0.4472136
Individual 4,-0.4472136,1.0954451,-0.4472136
Individual 5,-0.4472136,-0.7302967,1.7888544


In [20]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Big Sur ... 10.16

Matrix products: default
BLAS/LAPACK: /Users/serenadong/.pixi/envs/r-base/lib/libopenblasp-r0.3.29.dylib;  LAPACK version 3.12.0

locale:
[1] C

time zone: America/New_York
tzcode source: system (macOS)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] digest_0.6.37     IRdisplay_1.1     base64enc_0.1-3   fastmap_1.2.0    
 [5] glue_1.8.0        htmltools_0.5.8.1 repr_1.1.7        lifecycle_1.0.4  
 [9] cli_3.6.4         vctrs_0.6.5       pbdZMQ_0.3-13     compiler_4.3.3   
[13] tools_4.3.3       evaluate_1.0.3    pillar_1.10.2     rlang_1.1.5      
[17] jsonlite_1.9.1    crayon_1.5.3      IRkernel_1.3.2    uuid_1.2-1       